In [1]:
pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 10.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 11.6 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:0031m12.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.6 MB/s eta 0:00:0031m12.2 MB/s e

In [2]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

/home/yaochung41/github/t5-streamlit/t5-streamlit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from datasets import Dataset, DatasetDict

# Create a dictionary containing the questions and answers
data_dict = {
    "question": [
        "What is your name?",
        "How old are you?",
        "Where are you from?"
    ],
    "answer": [
        "My name is John.",
        "I am 30 years old.",
        "I am from New York."
    ]
}

# Create a dataset from the dictionary
dataset = Dataset.from_dict(data_dict)

# Split the dataset into train and test sets
train_dataset = dataset.select([0, 1])
test_dataset = dataset.select([2])

# Create a DatasetDict
dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset})


In [8]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1
    })
})

In [9]:
base = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(base)
tokenizer = T5Tokenizer.from_pretrained(base)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/home/yaochung41/github/t5-streamlit/t5-streamlit/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing c

In [23]:
# prefix our task
prefix1 = "rationale + "
prefix2 = "label + "

# define preprocessing function
def preprocess_function(examples):
  """Add prefix to the sentences, tokenize the text, and set the labels"""
  # the "inputs" are the tokenized answer
  inputs = [prefix1 + doc for doc in examples["question"]]
  model_inputs = tokenizer(inputs, max_length=128, truncation=True)

  # the "labels" are the tokenized outputs
  labels = [prefix2 + doc for doc in examples["answer"]]
  labels_token = tokenizer(labels, max_length=512, truncation=True)
  model_inputs["labels"] = labels_token["input_ids"]
  return model_inputs

# Map the preprocessing function across our datasets
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)


Map: 100%|██████████| 1/1 [00:00<00:00, 258.30 examples/s]


In [25]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})

In [12]:
# Use Rougue score to evaluate the training process
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
  preds, labels = eval_preds

  #decode preds and labels
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #rougeLSum expects newline after each sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  return result

In [13]:
# set up training arguments: for GPU with 16GB VRAM
training_args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    push_to_hub=False
)

In [26]:
# set up trainer
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [27]:
trainer.train()

 50%|█████     | 1/2 [00:01<00:01,  1.84s/it]/home/yaochung41/github/t5-streamlit/t5-streamlit/lib/python3.9/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                             
 50%|█████     | 1/2 [00:02<00:01,  1.84s/it]

{'eval_loss': 1.3975639343261719, 'eval_rouge1': 0.1818181818181818, 'eval_rouge2': 0.0, 'eval_rougeL': 0.1818181818181818, 'eval_rougeLsum': 0.1818181818181818, 'eval_runtime': 0.5976, 'eval_samples_per_second': 1.673, 'eval_steps_per_second': 1.673, 'epoch': 1.0}


                                             
100%|██████████| 2/2 [00:04<00:00,  2.32s/it]

{'eval_loss': 1.0151854753494263, 'eval_rouge1': 0.1818181818181818, 'eval_rouge2': 0.0, 'eval_rougeL': 0.1818181818181818, 'eval_rougeLsum': 0.1818181818181818, 'eval_runtime': 0.5788, 'eval_samples_per_second': 1.728, 'eval_steps_per_second': 1.728, 'epoch': 2.0}
{'train_runtime': 4.6405, 'train_samples_per_second': 0.862, 'train_steps_per_second': 0.431, 'train_loss': 1.6592155694961548, 'epoch': 2.0}


TrainOutput(global_step=2, training_loss=1.6592155694961548, metrics={'train_runtime': 4.6405, 'train_samples_per_second': 0.862, 'train_steps_per_second': 0.431, 'train_loss': 1.6592155694961548, 'epoch': 2.0})